OpenAI Agents SDK FrameWork - Sales Agents

Details: <a href="https://openai.github.io/openai-agents-python/">https://openai.github.io/openai-agents-python/</a>

In [141]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

# rich is a library for making formatted text output in the terminal
from rich.console import Console
import json
from openai import OpenAI
from IPython.display import Markdown, display

In [ ]:
load_dotenv(override=True)

In [ ]:
# Print the key prefixes to help with any debugging

openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')
groq_api_key = os.getenv('GROQ_API_KEY')
sendgrid_api_key = os.getenv('SENDGRID_API_KEY')
sendgrid_mailfrom = os.getenv('SENDGRID_MAILFROM')
sendgrid_mailto = os.getenv('SENDGRID_MAILTO')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set (and this is optional)")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:2]}")
else:
    print("Google API Key not set (and this is optional)")

if deepseek_api_key:
    print(f"DeepSeek API Key exists and begins {deepseek_api_key[:3]}")
else:
    print("DeepSeek API Key not set (and this is optional)")

if groq_api_key:
    print(f"Groq API Key exists and begins {groq_api_key[:4]}")
else:
    print("Groq API Key not set (and this is optional)")

if sendgrid_api_key:
    print(f"SendGrid API Key exists and begins {sendgrid_api_key[:4]}")
else:
    print("SendGrid API Key not set")

if sendgrid_mailfrom:
    print(f"MailFrom Key exists and begins {sendgrid_mailfrom[:4]}")
else:
    print("MailFrom  Key not set")

if sendgrid_mailto:
    print(f"MailTo Key exists and begins {sendgrid_mailto[:4]}")
else:
    print("MailTo Key not set")

In [ ]:
# Let's just check emails are working for you. 202 responce means delivery OK

def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email(os.environ.get('SENDGRID_MAILFROM'))  # Change to your verified sender
    to_email = To(os.environ.get('SENDGRID_MAILTO'))  # Change to your recipient
    content = Content("text/plain", "Your test email delivered")
    mail = Mail(from_email, to_email, "Test email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)

send_test_email()

In [ ]:
# Make an agent with name, instructions, model

agent = Agent(name="JokeTeller", instructions="You are a joke teller", model="gpt-4o-mini")

In [ ]:
agent

In [ ]:
# Run the joke with Runner.run(agent, prompt) then print final_output

with trace("JokerTeller Outputs"):
    result = await Runner.run(agent, "Tell a random joke about artificial Intelligence")
    print(result.final_output)

In [ ]:
openai = OpenAI()

In [ ]:
# Some lists!

todos = []
completed = []

In [ ]:
def show(text):
    try:
        Console().print(text)
    except Exception:
        print(text)

In [ ]:
def get_todo_report() -> str:
    result = ""
    for index, todo in enumerate(todos):
        if completed[index]:
            result += f"Todo #{index + 1}: [green][strike]{todo}[/strike][/green]\n"
        else:
            result += f"Todo #{index + 1}: {todo}\n"
    show(result)
    return result

In [ ]:
get_todo_report()

In [ ]:
def create_todos(descriptions: list[str]) -> str:
    todos.extend(descriptions)
    completed.extend([False] * len(descriptions))
    return get_todo_report()

In [ ]:
def mark_complete(index: int, completion_notes: str) -> str:
    if 1 <= index <= len(todos):
        completed[index - 1] = True
    else:
        return "No todo at this index."
    Console().print(completion_notes)
    return get_todo_report()

In [ ]:
todos, completed = [], []

create_todos(["Buy Tools", "Buy Fish", "Buy Banana"])

In [ ]:
mark_complete(1, "bought")

In [ ]:
create_todos_json = {
    "name": "create_todos",
    "description": "Add new todos from a list of descriptions and return the full list",
    "parameters": {
        "type": "object",
        "properties": {
            "descriptions": {
                'type': 'array',
                'items': {'type': 'string'},
                'title': 'Descriptions'
                }
            },
        "required": ["descriptions"],
        "additionalProperties": False
    }
}

In [ ]:
mark_complete_json = {
    "name": "mark_complete",
    "description": "Mark complete the todo at the given position (starting from 1) and return the full list",
    "parameters": {
        'properties': {
            'index': {
                'description': 'The 1-based index of the todo to mark as complete',
                'title': 'Index',
                'type': 'integer'
                },
            'completion_notes': {
                'description': 'Notes about how you completed the todo in rich console markup',
                'title': 'Completion Notes',
                'type': 'string'
                }
            },
        'required': ['index', 'completion_notes'],
        'type': 'object',
        'additionalProperties': False
    }
}

In [ ]:
tools = [{"type": "function", "function": create_todos_json},
        {"type": "function", "function": mark_complete_json}]

In [ ]:
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool","content": json.dumps(result),"tool_call_id": tool_call.id})
    return results

In [ ]:
def loop(messages):
    done = False
    while not done:
        response = openai.chat.completions.create(model="gpt-5.2", messages=messages, tools=tools, reasoning_effort="none")
        finish_reason = response.choices[0].finish_reason
        if finish_reason=="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results = handle_tool_calls(tool_calls)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
    show(response.choices[0].message.content)

In [ ]:
system_message = """
You are given a list of subjects to generate jokes for, by using your todo tools to plan a list of subjects, then carrying out each step in turn.
Now use the todo list tools, create a random joke for each subject, carry out the steps, and reply with the joke.
If any quantity isn’t provided in the question, then include a step to come up with a reasonable few subjects yourself.
Print your completion notes. Provide your jokes in Rich console markup without code blocks.
Do not ask the user questions or clarification; respond only with the answer after using your tools.
"""
user_message = """"
I'm interested in jokes for marketing, architecure, daily life and ai areas.
"""
messages = [{"role": "system", "content": system_message}, {"role": "user", "content": user_message}]

In [ ]:
todos, completed = [], []
loop(messages)

Agent Wrokflows

In [68]:
instructions1 = "You are a sales agent working for ADATUM, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ADATUM, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ADATUM, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [69]:
sales_agent1 = Agent(
        name="Professional Sales Agent",
        instructions=instructions1,
        model="gpt-4o-mini"
)

sales_agent2 = Agent(
        name="Engaging Sales Agent",
        instructions=instructions2,
        model="gpt-4o-mini"
)

sales_agent3 = Agent(
        name="Busy Sales Agent",
        instructions=instructions3,
        model="gpt-4o-mini"
)

In [71]:
result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Streamline Your SOC 2 Compliance Process with ADATUM

Dear [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I represent ADATUM, a cutting-edge SaaS solution designed to simplify and enhance the SOC 2 compliance process.

Navigating the complexities of compliance can often be a daunting task, especially when preparing for audits. With our AI-powered platform, we help organizations like yours automate compliance workflows, reduce manual efforts, and ensure rigorous adherence to SOC 2 requirements. 

Here are a few ways ADATUM can benefit your organization:

1. **Automated Documentation**: Generate and maintain necessary documentation effortlessly, ensuring you meet compliance demands in real time.
2. **Real-time Monitoring**: Stay updated with automated reports and alerts that keep you on track and ready for audits whenever needed.
3. **Guided Workflows**: Our intuitive interface guides your team step-by-step, reducing the learning curve and a

In [72]:
message = "Write a cold sales email"

with trace("Parallel cold emails"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")

Subject: Streamline Your SOC2 Compliance with ADATUM

Hi [Recipient's Name],

I hope this message finds you well. I'm reaching out to introduce you to ADATUM, our innovative SaaS solution designed to simplify the SOC2 compliance process and streamline audit preparations.

In today’s regulatory landscape, maintaining compliance can be both time-consuming and complex. Our AI-powered platform not only automates key processes but also enhances visibility into your compliance efforts, significantly reducing the risk of audit failures.

Key benefits of ADATUM include:

- **Automated Documentation**: Generate and manage necessary compliance documents effortlessly.
- **Real-Time Insights**: Monitor compliance status and risks with a single dashboard.
- **Seamless Collaboration**: Enable your team to work efficiently toward compliance goals.

Many organizations see the return on investment in both time and resources when leveraging ADATUM for their compliance needs. I would love to schedule a b

In [73]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You pick the best cold sales email from the given options. \
Imagine you are a customer and pick the one you are most likely to respond to. \
Do not give an explanation; reply with the selected email only.",
    model="gpt-4o-mini"
)

In [78]:
emails = "Cold sales emails:\n\n" + "\n\nEmail:\n\n".join(outputs)

best = await Runner.run(sales_picker, emails)

print(f"Best sales email:\n{best.final_output}")

Best sales email:
Subject: Turn SOC 2 Compliance from a Headache to a Hopscotch!

Hi [Recipient's Name],

Ever feel like getting SOC 2 compliant is akin to trying to solve a Rubik's Cube blindfolded? We get it—it's a complex puzzle that can turn even the coolest cucumber into a stressed-out pickle! 🥒

At ADATUM, we believe compliance shouldn't feel like an Olympic sport. Our AI-powered tool is here to transform the grueling audit grind into a smooth, easygoing stroll through the park—maybe even with a hot dog in hand! 🌭

Imagine a world where:
- You can automate compliance tasks faster than your morning coffee brews.
- Your audit prep becomes as easy as pie—without the messy clean-up!
- You can finally focus on what really matters: growing your business!

If you're ready to turn that compliance frown upside down, let’s chat! I promise—I won’t bring up spreadsheets in our conversation. 😉

Best,  
[Your Name]  
[Your Title]  
ADATUM  
[Your Contact Information]  

P.S. I’d love to share 

In [ ]:
message = "Write a cold sales email"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )
    outputs = [result.final_output for result in results]

    emails = "Cold sales emails:\n\n" + "\n\nEmail:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"Best sales email:\n{best.final_output}")

Check out the trace: https://platform.openai.com/traces

### Use of tools. 
All that json boilerplate and the `handle_tool_calls()` function with the if logic.

Simply wrap your function with the decorator `@function_tool`
This automatically converts into a tool, with the boilerplate json created


In [79]:
@function_tool
def send_email(body: str):
    """ Send out an email with the given body to all sales prospects """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email(os.environ.get('SENDGRID_MAILFROM'))  # Change to your verified sender
    to_email = To(os.environ.get('SENDGRID_MAILTO'))  # Change to your recipient
    content = Content("text/plain", body)
    mail = Mail(from_email, to_email, "Sales email", content).get()
    sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}

In [106]:
# Let's look at it
send_email.params_json_schema

{'properties': {'body': {'title': 'Body', 'type': 'string'}},
 'required': ['body'],
 'title': 'send_email_args',
 'type': 'object',
 'additionalProperties': False}

In [107]:
### convert an Agent into a tool
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")

In [108]:
tool1.params_json_schema

{'properties': {'input': {'title': 'Input', 'type': 'string'}},
 'required': ['input'],
 'title': 'sales_agent1_args',
 'type': 'object',
 'additionalProperties': False}

In [109]:
#A tool for each of our 3 email-writing agents
#And a tool for our function to send emails

description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, send_email]

In [113]:
tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000002037F7D0930>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000002037F7D0300>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='sales_agent3', descr

In [114]:
# Sales Manager callig planning agent tools

instructions = """
You are a Sales Manager at ADATUM. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
 
3. Use the send_email tool to send the best email (and only the best email) to the user.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
"""


sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)

### Add Some customisations

In [115]:
# Make an agent with name, instructions, model

agent_qa = Agent(name="QustionsAnswers", instructions="You are a enterpise solutions architect for corporate business unit", model="gpt-4o-mini")

In [117]:
request = "Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. "
request += "Answer only with the question, no explanation."
messages = [{"role": "user", "content": request}]

In [130]:
##request
messages

[{'role': 'user',
  'content': 'Please come up with a challenging, nuanced question that I can ask a number of LLMs to evaluate their intelligence. Answer only with the question, no explanation.'}]

In [ ]:
# Run the joke with Runner.run(agent, prompt) then print final_output

with trace("QA Agent Outputs"):
    result = await Runner.run(agent_qa, messages)
    print(result.final_output)

How can the ethical implications of deploying AI in decision-making processes be balanced with the need for efficiency and innovation in corporate environments?


In [135]:
competitors = []
answers = []
messages = [{"role": "user", "content": result.final_output}]

In [136]:
messages

[{'role': 'user',
  'content': 'How can the ethical implications of deploying AI in decision-making processes be balanced with the need for efficiency and innovation in corporate environments?'}]

In [195]:
competitors = []
answers = []
emails = []

model_name = "gpt-5-nano"
answer = "First Answer"
email = "test1@test.com"
competitors.append(model_name)
answers.append(answer)
emails.append(email)

model_name = "gemini-2.5-flash"
answer = "Secondt Answer"
email = "test2@test.com"
competitors.append(model_name)
answers.append(answer)
emails.append(email)

In [196]:
#display(Markdown(competitor))
competitors 
#display(Markdown(answer))
#answers
emails

['test1@test.com', 'test2@test.com']

In [198]:
# It's nice to know how to use "zip"
for competitor, answer in zip(competitors, emails):
    print(f"Competitor: {competitor}:{email}")

Competitor: gpt-5-nano:test2@test.com
Competitor: gemini-2.5-flash:test2@test.com


In [ ]:
### convert an Agent into a tool
tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description="Write a cold sales email")

In [206]:
result = Runner.run_streamed(sales_agent1, input=f"Write a cold sales email for Recipient {competitor} and use your name as  {email}")
async for event in result.stream_events():
    if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end="", flush=True)

Subject: Streamline Your SOC 2 Compliance with ADATUM

Dear Gemini Team,

I hope this message finds you well.

I’m reaching out to introduce you to ADATUM, an innovative SaaS solution designed to simplify SOC 2 compliance and expedite your audit preparation process. In today’s fast-paced environment, ensuring the security and integrity of your data is paramount, and ADATUM leverages the power of AI to make this easier than ever.

Our platform provides a comprehensive approach to compliance management, offering features such as:

- Automated documentation management
- Continuous monitoring and real-time reporting
- Customized workflows for audit readiness

By using ADATUM, companies like yours have reduced their audit preparation time by up to 50%, enabling teams to focus on what truly matters—growing their business.

I would love the opportunity to discuss how ADATUM can help Gemini streamline your compliance efforts. Would you be available for a brief call next week?

Thank you for co

In [212]:
@function_tool
def show(text: str):
    try:
        Console().print(text)
    except Exception:
        print(text)

In [213]:
#A tool for each of our 3 email-writing agents
#And a tool for our function to send emails

description = "Write a cold sales email"

tool1 = sales_agent1.as_tool(tool_name="sales_agent1", tool_description=description)
tool2 = sales_agent2.as_tool(tool_name="sales_agent2", tool_description=description)
tool3 = sales_agent3.as_tool(tool_name="sales_agent3", tool_description=description)

tools = [tool1, tool2, tool3, show]

In [214]:
tools

[FunctionTool(name='sales_agent1', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent1_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000002037FA747D0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='sales_agent2', description='Write a cold sales email', params_json_schema={'properties': {'input': {'title': 'Input', 'type': 'string'}}, 'required': ['input'], 'title': 'sales_agent2_args', 'type': 'object', 'additionalProperties': False}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x000002037FA749E0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None),
 FunctionTool(name='sales_agent3', descr

In [215]:
# Sales Manager callig planning agent tools

instructions = """
You are a Sales Manager at ADATUM. Your goal is to find the single best cold sales email using the sales_agent tools.
 
Follow these steps carefully:
1. Generate Drafts: Use all three sales_agent tools to generate three different email drafts. Do not proceed until all three drafts are ready.
 
2. Evaluate and Select: Review the drafts and choose the single best email using your judgment of which one is most effective.
 
3. Use the show tool to print the best email (and only the best email) to the user.
 
Crucial Rules:
- You must use the sales agent tools to generate the drafts — do not write them yourself.
- You must send ONE email using the send_email tool — never more than one.
"""


sales_manager = Agent(name="Sales Manager", instructions=instructions, tools=tools, model="gpt-4o-mini")

message = "Send a cold sales email addressed to 'Dear CEO'"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message)

Subject: Let’s Make SOC2 Compliance as Fun as a Team Building Retreat! 🎉

Hey [CEO's Name],

I know what you’re thinking: “Another email about compliance? Yawn!” But hold on to your artisanal coffee — this 
isn’t your ordinary compliance chat.

At ADATUM, we’ve taken the stress out of SOC2 compliance and audits by harnessing the power of AI. Think of us as 
the personal trainer for your compliance journey — we whip your operations into shape without making you do a 
single burpee! 

Our clients are turning compliance from a stressful slog into a walk in the park (with snacks, of course). 💼➡️🌳 
But don't just take my word for it; I’d love to share some amazing success stories over a quick chat this week. 

Ready to transform the way you think about compliance? Let’s set up a time — I promise to bring the witty banter!

Cheers,  
[Your Name]  
Sales Manager, ADATUM  
P.S. I hear compliance counts as a cardio workout, so you’re getting your steps in, too! 😂

In [227]:

# ===== Imports & State =====
from typing import List
from rich.console import Console


In [228]:

# If your framework provides this decorator, import it from there
# from agents.tools import function_tool
# from agents.core import Agent, Runner, trace

# --- In-memory store for todos ---
todos: List[str] = []
completed: List[bool] = []


In [233]:
# ----- Tools (decorator without kwargs) -----
@function_tool
def show(text: str):
    """Display text to the user."""
    try:
        Console().print(text)
    except Exception:
        print(text)


In [235]:

def _build_todo_report() -> str:
    if not todos:
        return "To-do list is empty."
    lines = ["🧾 To‑do List:"]
    for i, (desc, done) in enumerate(zip(todos, completed), start=1):
        checkbox = "✅" if done else "⬜"
        lines.append(f"{i}. {checkbox} {desc}")
    return "\n".join(lines)


In [236]:

@function_tool
def get_todo_report() -> str:
    """Get a formatted report of current to-dos."""
    return _build_todo_report()


In [237]:

@function_tool
def create_todos(descriptions: List[str]) -> str:
    """Create todo items from a list of descriptions and return a report."""
    todos.extend(descriptions)
    completed.extend([False] * len(descriptions))
    return _build_todo_report()


In [244]:

instructions = """
You are an agent at ADATUM. Generate exactly five random shopping to-do items using the tools.

Steps:
1) Call "create_todos" with one argument "descriptions": a JSON array of exactly 5 short shopping items, e.g.
   ["bread", "milk", "eggs", "coffee", "fruit"].

2) Take the returned report string and call "show" with that string in "text" to display it.

Constraints:
- Do NOT write the list yourself; items must be created via create_todos.
- Do NOT reformat the report; display exactly what create_todos returns.
- End after calling show.
"""

sales_manager = Agent(
    name="Sales Manager",
    instructions=instructions,
    model="gpt-4o-mini"
)


message = "Print to do list"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message, max_turns=12)

print("\n--- Agent Final Output ---")
print(result)



--- Agent Final Output ---
RunResult:
- Last agent: Agent(name="Sales Manager", ...)
- Final output (str):
    Here are your shopping to-do items:
    
    - pasta
    - olive oil
    - cheese
    - salad
    - soda
    
- 3 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


In [243]:
get_todo_report

FunctionTool(name='get_todo_report', description='Get a formatted report of current to-dos.', params_json_schema={'properties': {}, 'title': 'get_todo_report_args', 'type': 'object', 'additionalProperties': False, 'required': []}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000020300A877F0>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)

In [258]:


from typing import List
from rich.console import Console

# In-memory state
todos: List[str] = []
completed: List[bool] = []

# ---------- Plain Python helpers (safe to call directly) ----------

def show_impl(text: str) -> str:
    """Render text with Rich (if available) and return it."""
    try:
        Console().print(text, soft_wrap=True, markup=True)
    except Exception:
        print(text)
    return text

def build_todo_report() -> str:
    """Return a Rich-markup string for the current todo list."""
    if not todos:
        return "To-do list is empty."
    lines = []
    for index, todo in enumerate(todos):
        if completed[index]:
            lines.append(f"Todo #{index + 1}: [green][strike]{todo}[/strike][/green]")
        else:
            lines.append(f"Todo #{index + 1}: {todo}")
    return "\n".join(lines)

def get_todo_report_impl() -> str:
    """Build and show the report, then return it."""
    result = build_todo_report()
    show_impl(result)
    return result

def create_todos_impl(descriptions: List[str]) -> str:
    """Add todos and return a ready-to-render report."""
    todos.extend(descriptions)
    completed.extend([False] * len(descriptions))
    return build_todo_report()

# ---------- Tool wrappers (for the agent runtime) ----------

@function_tool
def show(text: str) -> str:
    """Tool: Display text to the user using Rich and return it."""
    return show_impl(text)

@function_tool
def get_todo_report() -> str:
    """Tool: Build the to-do report, display it, and return the string."""
    return get_todo_report_impl()


@function_tool
def create_todos(descriptions: List[str]) -> str:
    todos.extend(descriptions)
    completed.extend([False] * len(descriptions))
    return _build_todo_report()   # <-- use your Rich 



In [267]:

# Seed some data
todos[:] = ["milk", "bread", "eggs", "coffee", "fruit"]
completed[:] = [False, True, False, False, True]

# Call the helper (not the tool)
report = get_todo_report_impl()
#print("--- Returned string (for debug) ---")
#print(report)



Todo #1: milk
Todo #2: bread
Todo #3: eggs
Todo #4: coffee
Todo #5: fruit

In [273]:

instructions = """
You are an agent at ADATUM. Generate exactly five random shopping to‑do items using the tools.

Steps:

1) Call the tool "create_todos" with one argument:
   "descriptions": a JSON array of exactly five short shopping items, for example:
   ["bread", "milk", "eggs", "coffee", "fruit"].

2) Immediately call the tool "show" with exactly the string returned by create_todos.


Constraints:
- Do NOT write the list yourself; items must be created via create_todos.
- Do NOT modify or reformat the report; show exactly what create_todos returns.
- Do NOT call get_todo_report_impl (it is not a tool).
- Finish the task after calling show.
"""

sales_manager = Agent(
    name="Sales Manager",
    instructions=instructions,
    model="gpt-4o-mini"
)

message = "Print to do list"

with trace("Sales manager"):
    result = await Runner.run(sales_manager, message, max_turns=12)

print("\n--- Agent Final Output ---")
print(result)



--- Agent Final Output ---
RunResult:
- Last agent: Agent(name="Sales Manager", ...)
- Final output (str):
    {"descriptions":["toothpaste","bananas","chicken breasts","olive oil","notebook"]}
- 2 new item(s)
- 1 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


In [276]:
reset_todos 

FunctionTool(name='reset_todos', description='', params_json_schema={'properties': {}, 'title': 'reset_todos_args', 'type': 'object', 'additionalProperties': False, 'required': []}, on_invoke_tool=<function function_tool.<locals>._create_function_tool.<locals>._on_invoke_tool at 0x0000020300CA8880>, strict_json_schema=True, is_enabled=True, tool_input_guardrails=None, tool_output_guardrails=None)